# Training a model to identify watches

Data: scraped from ebay listings

Categories: ['rolex','omega','seiko','invicta','hamilton','tissot','bulova','timex','elgin','unbranded/other']

In [11]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
import pandas as pd
import csv
from PIL import Image
from numpy import asarray

# Data preparation

In [45]:
def get_image(image_name):
    image = Image.open(f"../training_data/{image_name}.jpg")
    img_data = asarray(image).astype('float32')
    return img_data

In [46]:
#read in csv to pandas df
df = pd.read_csv('../data_management/watch_labels.csv', sep=',',names=['name', 'type', 'link'])
#drop the link column and add image column (npy image)
df.drop(['link'], axis=1,inplace=True)
df["image"] = df["name"].apply(get_image)
df.head()

,name,type,image
0,watch0,rolex,"[[[238.0, 236.0, 237.0], [238.0, 236.0, 237.0]..."
1,watch1,rolex,"[[[234.0, 236.0, 233.0], [234.0, 236.0, 233.0]..."
2,watch2,rolex,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]..."
3,watch3,rolex,"[[[37.0, 36.0, 41.0], [37.0, 36.0, 41.0], [37...."
4,watch4,rolex,"[[[234.0, 236.0, 233.0], [234.0, 236.0, 233.0]..."


In [47]:
print(df.iloc[3]['image'].shape)

(300, 200, 3)


In [49]:
#convert to npy arrays for training and test
X = df['image'].to_numpy()
Y = df['type'].to_numpy()
print(X.shape,Y.shape)

x_train = X[0:60000]
y_train = Y[0:60000]
x_test = X[60000:]
y_test = Y[60000:]


(73270,) (73270,)


In [51]:
i_shape = (300, 200, 3)

model = Sequential([
    Conv2D(filters=20, kernel_size=(5,5), strides=(1, 1), input_shape=[28,28,1], padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=20, kernel_size=(5,5), strides=(1, 1), input_shape=[14,14,1], padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Flatten(), # flattens into one vector
    keras.layers.Dropout(0.5),
    Dense(num_classes, activation="softmax") #the 10 ouput neurons
])

model.summary()
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 300, 200, 20)      1520      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 150, 100, 20)     0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 150, 100, 20)      10020     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 75, 50, 20)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 75000)             0         
                                                                 
 dropout_5 (Dropout)         (None, 75000)            

[None]